In [12]:
# autor: Łukasz Machnik
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

In [13]:
numberOfProducts = 4
numberOfMachines = 3
availableTime = 3600
productDemand = [400, 100, 150, 500]
productPrice = [9, 7, 6, 5] # per kg
materialCost = [4, 1, 1, 1] # per kg
operatingCost = [2, 2, 3] # per min
productEarning = [productPrice[i] - materialCost[i] for i in range(numberOfProducts)]
productionTime = [[5, 10, 6], [3, 6, 4], [4, 5, 3], [4, 2, 1]] # min per kg


In [14]:
model = LpProblem(name="zyski_ze_sprzedazy", sense=LpMaximize)
unitsMade = [LpVariable("wytworzono_produkt%s" % i, lowBound=0) for i in range(numberOfProducts)]
occupiedTime = [0 for j in range(numberOfMachines)]
for i in range(numberOfProducts):
    for j in range(numberOfMachines):
        occupiedTime[j] += unitsMade[i] * productionTime[i][j]
for j in range(numberOfMachines):
    model += (occupiedTime[j] <= availableTime, "ograniczenie_czasu%s" % j)
for i in range(numberOfProducts):
    model += (unitsMade[i] <= productDemand[i], "popyt%s" % i)
totalEarnings = 0
for i in range(numberOfProducts):
    totalEarnings += unitsMade[i] * productEarning[i]
for j in range(numberOfMachines):
    totalEarnings -= occupiedTime[j] * operatingCost[j] / 60.0
model += totalEarnings
model.solve()

1

In [15]:
# model = LpProblem(name="produkcja_towarow", sense=LpMaximize)
# unitsMade = [[LpVariable(("produkt%s_maszyna%s" % (i, j)), lowBound=0) for j in range(numberOfMachines)] for i in range(numberOfProducts)]
# occupiedTime = [0 for i in range(numberOfMachines)]

# for i in range(numberOfProducts):
#     model += (lpSum(unitsMade[i]) <= productDemand[i], "popyt%s" % i)

# for j in range(numberOfMachines):
#     for i in range(numberOfProducts):
#         occupiedTime[j] += unitsMade[i][j] * productionTime[i][j]
#     model += (occupiedTime[j] <= availableTime, "ograniczenie_czasu%s" % j)

# totalEarnings = 0
# for i in range(numberOfProducts):
#     for j in range(numberOfMachines):
#         totalEarnings += unitsMade[i][j] * productEarning[i]
# for j in range(numberOfMachines):
#     totalEarnings -= occupiedTime[j] * operatingCost[j] / 60.0
# model += totalEarnings
# # print(model)
# status = model.solve()# 

In [16]:
print("Całkowity zysk: %s" % model.objective.value())
for var in model.variables():
    print("%s = %s" % (var.name, var.value()))

Całkowity zysk: 3632.5
wytworzono_produkt0 = 125.0
wytworzono_produkt1 = 100.0
wytworzono_produkt2 = 150.0
wytworzono_produkt3 = 500.0
